# Fantasy Football WAR Calculator - Example Usage

This notebook demonstrates how to use the Fantasy Football WAR (Wins Above Replacement) calculator with MPPR (Minus PPR) scoring.

## Overview

The WAR system calculates how many wins each player contributes above a replacement-level player at their position. This is based on:

- **MPPR Scoring**: EPA-based scoring system with negative points for attempts/targets
- **Replacement Level**: The worst startable player at each position
- **Win Probability**: Normal distribution-based probability calculations
- **Auction Values**: Dollar values based on WAR calculations

In [ ]:
# Install required packages if running in a new environment
# !pip install -e .

import polars as pl  # Using polars instead of pandas!
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from loguru import logger

# Fantasy WAR imports
from fantasy_war.config.leagues import LeagueConfig, fal_league
from fantasy_war.config.scoring import mppr_scoring
from fantasy_war.data.loaders import NFLDataLoader
from fantasy_war.data.processors import StatsProcessor
from fantasy_war.calculators.war_engine import WARCalculator
from fantasy_war.calculators.auction_values import AuctionValueCalculator
from fantasy_war.data.cache import cache_manager

# Configure plotting
plt.style.use('default')  # Using default style for compatibility
sns.set_palette("husl")

logger.info("Fantasy WAR system initialized")

## 1. Configure Your League Settings

First, let's set up the league configuration with your specific roster requirements and scoring system.

In [ ]:
# Create custom league configuration
my_league = LeagueConfig(
    name="My Fantasy League",
    teams=16,  # Adjust to your league size
    regular_season_weeks=list(range(1, 13)),  # Weeks 1-12
    playoff_weeks=list(range(13, 18)),  # Weeks 13-17
    minimum_games_played=1,  # Minimum games for WAR eligibility
)

print(f"League: {my_league.name}")
print(f"Teams: {my_league.teams}")
print(f"Regular Season: Weeks {my_league.regular_season_weeks}")
print(f"Playoff Weeks: {my_league.playoff_weeks}")
print(f"Total Starters: {my_league.roster.total_starters}")

# Show roster requirements
print("\nRoster Requirements:")
for position in ['QB', 'RB', 'WR', 'TE', 'PK', 'PN', 'DT', 'DE', 'LB', 'CB', 'S']:
    min_req, max_req = my_league.roster.get_position_requirements(position)
    if max_req > 0:
        print(f"  {position}: {min_req}-{max_req} starters")

## 2. Load NFL Data

Load play-by-play data and calculate player statistics using both Python (nfl_data_py) and R (nflfastR) sources.

In [ ]:
# Initialize data loader
data_loader = NFLDataLoader()

# Load data for recent seasons
seasons = [2022, 2023]  # Adjust as needed
logger.info(f"Loading NFL data for seasons: {seasons}")

# Load player statistics (this may take a few minutes on first run)
stats_df = data_loader.load_player_stats(seasons, weekly=True)

print(f"Loaded {len(stats_df)} player-week records")
print(f"Seasons: {sorted(stats_df['season'].unique())}")
print(f"Weeks: {sorted(stats_df['week'].unique())}")
print(f"Positions: {sorted(stats_df['position'].unique())}")

# Show sample data
stats_df.head()

## 3. Calculate Fantasy Points

Process the raw NFL statistics and calculate MPPR fantasy points for each player-week.

In [ ]:
# Initialize stats processor with league configuration
stats_processor = StatsProcessor(my_league, mppr_scoring)

# Calculate fantasy points
logger.info("Calculating MPPR fantasy points")
stats_with_points = stats_processor.calculate_fantasy_points(stats_df)

# Show fantasy points distribution
print(f"Fantasy points calculated for {len(stats_with_points)} player-weeks")
print(f"Average MPPR points per game: {stats_with_points['fantasy_points_mppr'].mean():.2f}")
print(f"Top single-game performance: {stats_with_points['fantasy_points_mppr'].max():.2f}")

# Show top single-game performances
top_games = (
    stats_with_points
    .sort_values('fantasy_points_mppr', ascending=False)
    .head(10)[['player_name', 'position', 'season', 'week', 'fantasy_points_mppr']]
)

print("\nTop 10 Single-Game Performances:")
top_games

## 4. Calculate WAR Values

This is the core of the system - calculating Wins Above Replacement for each player.

In [ ]:
# Initialize WAR calculator
war_calculator = WARCalculator(my_league)

# Calculate league-wide WAR for the most recent season
target_season = [2023]  # Focus on most recent complete season
target_weeks = list(range(1, 18))  # Full season

logger.info(f"Calculating WAR for season {target_season}, weeks {target_weeks}")

# This is the main calculation - may take a few minutes
league_war = war_calculator.calculate_league_war(
    stats_with_points,
    seasons=target_season,
    weeks=target_weeks
)

print(f"WAR calculation complete for {league_war.season}")
print(f"Positions analyzed: {len(league_war.position_results)}")
print(f"Total WAR generated: {league_war.total_war_generated:.2f}")

# Show WAR by position
print("\nWAR by Position:")
for position, avg_war in league_war.average_war_per_position.items():
    pos_result = league_war.position_results.get(position)
    if pos_result:
        total_war = sum(war.wins_above_replacement for war in pos_result.player_wars)
        print(f"  {position}: {total_war:.1f} total, {avg_war:.2f} average")

## 5. Analyze WAR Results

Let's examine the top performers and analyze the WAR distribution.

In [ ]:
# Get top 20 players overall
top_players = league_war.top_players_overall[:20]

# Create DataFrame for analysis (using Polars now!)
war_data = []
for war in top_players:
    war_data.append({
        'player_name': war.player_name,
        'position': war.position,
        'war': war.wins_above_replacement,
        'waa': war.wins_above_average,
        'games': war.games_played,
        'total_points': war.total_fantasy_points,
        'avg_points': war.average_fantasy_points,
        'win_pct': war.win_percentage
    })

# Convert to Polars DataFrame
top_df = pl.DataFrame(war_data)

print("Top 20 Players by WAR:")
print(top_df.with_columns(pl.all().round(2)))

# Convert to pandas for plotting (matplotlib expects pandas)
top_df_pandas = top_df.to_pandas()

# Create WAR visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Top players bar chart
ax1.barh(range(len(top_df_pandas)), top_df_pandas['war'], color='steelblue')
ax1.set_yticks(range(len(top_df_pandas)))
ax1.set_yticklabels([f"{row['player_name']} ({row['position']})" for _, row in top_df_pandas.iterrows()])
ax1.set_xlabel('Wins Above Replacement')
ax1.set_title('Top 20 Players by WAR')
ax1.invert_yaxis()

# WAR by position
position_war = top_df_pandas.groupby('position')['war'].sum().sort_values(ascending=False)
ax2.bar(position_war.index, position_war.values, color='lightcoral')
ax2.set_xlabel('Position')
ax2.set_ylabel('Total WAR')
ax2.set_title('WAR Distribution by Position (Top 20)')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 6. Calculate Auction Values

Convert WAR values into auction/draft dollar values for your fantasy draft.

In [ ]:
# Initialize auction calculator
auction_budget = 200  # Adjust to your league's budget
auction_calc = AuctionValueCalculator(my_league, auction_budget)

# Calculate auction values
logger.info(f"Calculating auction values with ${auction_budget} budget")
auction_values = auction_calc.calculate_league_auction_values(league_war)

print(f"Auction values calculated for {len(auction_values)} players")
print(f"Total auction value: ${sum(av.auction_value_dollars for av in auction_values):,.0f}")
print(f"Average $/WAR: ${league_war.dollars_per_war_league_average:.2f}")

# Top 30 auction values
top_values = sorted(auction_values, key=lambda x: x.auction_value_dollars, reverse=True)[:30]

auction_data = []
for av in top_values:
    auction_data.append({
        'player_name': av.player_name,
        'position': av.position,
        'auction_value': av.auction_value_dollars,
        'war': av.wins_above_replacement,
        'war_rank': av.war_rank_overall,
        'pos_rank': av.war_rank_position,
        'tier': av.draft_tier,
        'budget_pct': av.budget_percentage,
        'sleeper': '💎' if av.is_sleeper else '',
        'bust_risk': '⚠️' if av.is_bust_risk else ''
    })

auction_df = pd.DataFrame(auction_data)

print("\nTop 30 Auction Values:")
auction_df.round(2)

## 7. Draft Strategy Analysis

Analyze optimal budget allocation and identify value opportunities.

In [ ]:
# Generate draft board and budget analysis
draft_board = auction_calc.generate_draft_board(auction_values, sort_by="auction_value")
budget_allocation = auction_calc.calculate_optimal_budget_allocation(auction_values)

print("Optimal Budget Allocation by Position:")
print("-" * 60)
for position, analysis in budget_allocation.items():
    print(f"{position:3s}: ${analysis['top_tier_cost']:6.0f} (top {analysis['max_starters']}), "
          f"{analysis['recommended_budget_pct']:4.1f}% of budget, "
          f"{analysis['scarcity_rating']} scarcity")

# Identify sleepers and bust risks
sleepers = [av for av in auction_values if av.is_sleeper]
bust_risks = [av for av in auction_values if av.is_bust_risk]

print(f"\n💎 Sleeper Candidates ({len(sleepers)} found):")
for sleeper in sleepers[:10]:
    print(f"  {sleeper.player_name} ({sleeper.position}): ${sleeper.auction_value_dollars:.0f}, WAR: {sleeper.wins_above_replacement:.2f}")

print(f"\n⚠️ Potential Bust Risks ({len(bust_risks)} found):")
for bust in bust_risks[:5]:
    print(f"  {bust.player_name} ({bust.position}): ${bust.auction_value_dollars:.0f}, WAR: {bust.wins_above_replacement:.2f}")

# Value efficiency chart
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Auction value vs WAR scatter plot
for position in ['QB', 'RB', 'WR', 'TE']:
    pos_data = [av for av in auction_values if av.position == position]
    x_vals = [av.wins_above_replacement for av in pos_data]
    y_vals = [av.auction_value_dollars for av in pos_data]
    ax1.scatter(x_vals, y_vals, label=position, alpha=0.7, s=50)

ax1.set_xlabel('Wins Above Replacement')
ax1.set_ylabel('Auction Value ($)')
ax1.set_title('Auction Value vs WAR')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Budget allocation pie chart
position_budgets = [(pos, data['recommended_budget_pct']) for pos, data in budget_allocation.items()]
position_budgets = sorted(position_budgets, key=lambda x: x[1], reverse=True)[:8]  # Top 8 positions

positions, percentages = zip(*position_budgets)
ax2.pie(percentages, labels=positions, autopct='%1.1f%%', startangle=90)
ax2.set_title('Recommended Budget Allocation')

plt.tight_layout()
plt.show()

## 8. Export Results

Save your analysis results for draft preparation.

In [ ]:
# Export draft board to CSV
draft_df = pd.DataFrame(draft_board)
draft_df.to_csv('fantasy_war_draft_board.csv', index=False)
print("Draft board saved to: fantasy_war_draft_board.csv")

# Export detailed WAR results
war_results = []
for pos_result in league_war.position_results.values():
    for war in pos_result.player_wars:
        war_results.append({
            'player_id': war.player_id,
            'player_name': war.player_name,
            'position': war.position,
            'season': war.season,
            'games_played': war.games_played,
            'total_fantasy_points': war.total_fantasy_points,
            'average_fantasy_points': war.average_fantasy_points,
            'wins_above_replacement': war.wins_above_replacement,
            'wins_above_average': war.wins_above_average,
            'win_percentage': war.win_percentage,
            'expected_wins': war.expected_wins
        })

war_results_df = pd.DataFrame(war_results)
war_results_df = war_results_df.sort_values('wins_above_replacement', ascending=False)
war_results_df.to_csv('fantasy_war_detailed_results.csv', index=False)
print("Detailed WAR results saved to: fantasy_war_detailed_results.csv")

# Show cache statistics
cache_stats = cache_manager.get_stats()
print(f"\n📊 Cache Statistics:")
print(f"  Size: {cache_stats['size_mb']:.1f} MB ({cache_stats['count']} entries)")
print(f"  Directory: {cache_stats['directory']}")
print(f"  Next calculation will be faster thanks to caching!")

## 9. Position-Specific Analysis

Dive deeper into specific positions to understand value tiers and replacement levels.

In [ ]:
# Analyze a specific position (e.g., Running Backs)
position_to_analyze = 'RB'
rb_result = league_war.position_results.get(position_to_analyze)

if rb_result:
    print(f"\n🏃 {position_to_analyze} Position Analysis:")
    print(f"Total {position_to_analyze}s analyzed: {len(rb_result.player_wars)}")
    print(f"Replacement level rank: {rb_result.replacement_level_rank}")
    print(f"Starter spots league-wide: {rb_result.total_starter_spots}")
    
    # Top 20 RBs
    rb_data = []
    for i, war in enumerate(rb_result.player_wars[:20], 1):
        rb_data.append({
            'rank': i,
            'player': war.player_name or war.player_id,
            'war': war.wins_above_replacement,
            'games': war.games_played,
            'avg_pts': war.average_fantasy_points,
            'total_pts': war.total_fantasy_points
        })
    
    rb_df = pd.DataFrame(rb_data)
    print(f"\nTop 20 {position_to_analyze}s by WAR:")
    print(rb_df.round(2))
    
    # WAR distribution chart for this position
    war_values = [war.wins_above_replacement for war in rb_result.player_wars[:30]]
    
    plt.figure(figsize=(12, 6))
    
    plt.subplot(1, 2, 1)
    plt.plot(range(1, len(war_values) + 1), war_values, 'o-', linewidth=2, markersize=6)
    plt.axhline(y=0, color='red', linestyle='--', alpha=0.7, label='Replacement Level')
    plt.axvline(x=rb_result.replacement_level_rank, color='red', linestyle='--', alpha=0.7)
    plt.xlabel('Player Rank')
    plt.ylabel('Wins Above Replacement')
    plt.title(f'{position_to_analyze} WAR Drop-off Curve')
    plt.grid(True, alpha=0.3)
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.hist(war_values, bins=15, alpha=0.7, color='skyblue', edgecolor='black')
    plt.axvline(x=0, color='red', linestyle='--', alpha=0.7, label='Replacement Level')
    plt.xlabel('Wins Above Replacement')
    plt.ylabel('Number of Players')
    plt.title(f'{position_to_analyze} WAR Distribution')
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
else:
    print(f"No data found for position: {position_to_analyze}")

## 10. Summary and Draft Recommendations

Based on the WAR analysis, here are key insights for your draft strategy.

In [ ]:
# Generate draft strategy summary
print("🏆 Fantasy Draft Strategy Summary")
print("=" * 50)

# Tier 1 players (elite)
tier_1_players = [av for av in auction_values if av.draft_tier == 1]
tier_1_cost = sum(av.auction_value_dollars for av in tier_1_players)

print(f"\n💎 Tier 1 Elite Players ({len(tier_1_players)} players):")
print(f"   Total cost: ${tier_1_cost:.0f} ({tier_1_cost/auction_budget*100:.1f}% of budget)")
for player in sorted(tier_1_players, key=lambda x: x.auction_value_dollars, reverse=True)[:5]:
    print(f"   • {player.player_name} ({player.position}): ${player.auction_value_dollars:.0f}")

# Best values (highest WAR per dollar)
best_values = sorted(auction_values, 
                    key=lambda x: x.wins_above_replacement / x.auction_value_dollars if x.auction_value_dollars > 0 else 0,
                    reverse=True)[:10]

print(f"\n💡 Best Values (WAR per Dollar):")
for i, player in enumerate(best_values, 1):
    efficiency = player.wins_above_replacement / player.auction_value_dollars if player.auction_value_dollars > 0 else 0
    print(f"   {i:2d}. {player.player_name} ({player.position}): "
          f"${player.auction_value_dollars:.0f}, {efficiency:.3f} WAR/$")

# Position scarcity insights
print(f"\n📊 Position Scarcity Analysis:")
scarcity_order = sorted(budget_allocation.items(), 
                       key=lambda x: x[1]['recommended_budget_pct'], 
                       reverse=True)

for position, data in scarcity_order[:6]:  # Top 6 positions
    print(f"   {position}: {data['scarcity_rating']} scarcity, "
          f"invest {data['recommended_budget_pct']:.1f}% of budget")

print(f"\n🎯 Draft Strategy Recommendations:")
print(f"   1. Target elite players early - WAR drops off significantly after top tiers")
print(f"   2. Focus budget on high-scarcity positions with limited depth")
print(f"   3. Consider sleeper candidates for value in middle/late rounds")
print(f"   4. Avoid bust risks unless price drops significantly")
print(f"   5. Remember: WAR accounts for your league's specific scoring and roster settings")

# Final cache cleanup
print(f"\n🗄️ Analysis complete! Data cached for faster future calculations.")
print(f"   Run 'cache_manager.clear_all()' if you want to refresh all data.")

## Next Steps

Now that you have your WAR analysis and auction values:

1. **Review the draft board** - Use the CSV exports for your draft preparation
2. **Adjust for league specifics** - Consider your league's tendencies and biases
3. **Monitor injury news** - Update player values based on health status
4. **Track ADP vs WAR values** - Find players being drafted below their WAR value
5. **Re-run analysis** - Update calculations as new data becomes available

Use the CLI tool for quick calculations:
```bash
fantasy-war calculate-war --seasons 2023 --output war_results.csv
fantasy-war auction-values --budget 200 --output auction_values.csv
```

Good luck dominating your fantasy league! 🏆